In [ ]:
# Updating daily_adjusted table according to stock splits

######################################################
#COMPLETE THE FOLLOWING ACCORDING TO YOUR MYSQL DATABASE
h=''
u=''
db=''
pswrd=''
######################################################

In [52]:
import mysql.connector
from mysql.connector import Error
import pandas as pd

# SQL and basic setup

# Allowing users to find out the name of the symbol by choosing the name of the company
try:
    # connecting to SQL database
    mySQLconnection = mysql.connector.connect(host=h,
                             database=db,
                             user=u,
                             password=pswrd, autocommit=True)
    
    # Actual Query here
    
    # First, I get the range of dates for each stock and their corresponding names 
    sql_select_Query1 = "\
UPDATE daily_stock \
SET s_adj=0 \
WHERE split!=1 and s_adj=1"
    sql_select_Query2 = "CREATE TABLE T LIKE daily_stock "
    sql_select_Query3 = "INSERT T SELECT * FROM daily_stock"
    sql_select_Query4 ="Select count(daily_id) \
    from daily_stock \
    where s_adj=0"
    sql_select_Query5 = "DROP TABLE daily_stock"
    sql_select_Query6="RENAME TABLE T to daily_stock"
    
    
    sql_adj1 = "UPDATE T SET d_open = d_open / (Select split from s_ratio),\
d_close= d_close/ (Select split from s_ratio),d_high= d_high / (Select split from s_ratio),\
d_low= d_low / (Select split from s_ratio), d_volume= d_volume * (Select split from s_ratio) \
Where symbol = (Select symbol from s_ratio) and date < (Select date from s_ratio)"
    sql_adj2="UPDATE T SET s_adj=2 Where daily_id=(select daily_id from s_ratio)"
    sql_adj3="UPDATE daily_stock \
    SET s_adj=2 Where daily_id= (Select MIN(daily_id) from T where s_adj=2)"

    cursor1 = mySQLconnection.cursor()
    cursor2 = mySQLconnection.cursor()
    cursor3 = mySQLconnection.cursor()
    cursor4 = mySQLconnection.cursor()
    cursor5 = mySQLconnection.cursor()
    cursor6 = mySQLconnection.cursor()
    cursorA1 = mySQLconnection.cursor()
    cursorA2 = mySQLconnection.cursor()
    cursorA3 = mySQLconnection.cursor()
    
    cursor1.execute(sql_select_Query1)
    cursor2.execute(sql_select_Query2)
    cursor3.execute(sql_select_Query3)
    cursor4.execute(sql_select_Query4)
    c = cursor4.fetchall()
    count= (c[0][0])
    print("adjusting for "+count+" splits in data")
    while (count>0):
        cursorA1.execute(sql_adj1)
        cursorA2.execute(sql_adj2)
        cursorA3.execute(sql_adj3)
        count=count-1
    
    cursor5.execute(sql_select_Query5)
    cursor6.execute(sql_select_Query6)
    
    cursor1.close()
    cursor2.close()
    cursor3.close()
    cursor4.close()
    cursor6.close()
    cursor5.close()
    cursorA1.close()
    cursorA2.close()
    cursorA3.close()
except Error as e :
    print ("Error", e)
    
finally:
    #closing database connection.
    if(mySQLconnection.is_connected()):
        mySQLconnection.close()
        

131


In [ ]:
# getting technical indicators using the daily stock prices


try:
    # connecting to SQL database
    mySQLconnection = mysql.connector.connect(host=h,
                             database=db,
                             user=u,
                             password=pswrd, autocommit=True)
    
    
    
    df = pd.read_sql("Select * from daily_stock order by symbol,date",mySQLconnection)
except Error as e :
    print ("Error", e)
    
finally:
    #closing database connection.
    if(mySQLconnection.is_connected()):
        mySQLconnection.close()


s = df['symbol'].unique()
import numpy as np
df['sma10']=np.nan
df['sma30']=np.nan
df['sma50']=np.nan
df['ema10']=np.nan
df['ema30']=np.nan
df['ema50']=np.nan
df['macd']=np.nan
df['macd_signal']=np.nan
for x in s:
    temp = df.loc[(df['symbol']==x)]
    
    temp['sma10']= temp.d_close.rolling(window=10).mean()
    temp['sma30'] = temp.d_close.rolling(window=30).mean()
    temp['sma50'] = temp.d_close.rolling(window=50).mean()
    
    temp['ema10'] = temp.d_close.ewm(span=10, adjust=False, min_periods=10).mean()
    temp['ema30'] = temp.d_close.ewm(span=30, adjust=False, min_periods=30).mean()
    temp['ema50'] = temp.d_close.ewm(span=50, adjust=False, min_periods=50).mean()
    
    exp1 = temp.d_close.ewm(span=12, adjust=False, min_periods=12).mean()
    exp2 = temp.d_close.ewm(span=26, adjust=False, min_periods=26).mean()
    macd=exp1-exp2
    temp['macd']=macd
    temp['macd_signal']=macd.ewm(span=9, adjust=False, min_periods=9).mean()
    df.update(temp)
df.drop(columns=['d_open', 'd_close','d_high','d_low','d_volume','d_adjusted_close','split','s_adj','d_dividend'],inplace=True)
df.dropna(inplace=True)
df.to_csv("technical_indicators.csv")

In [270]:
# Inserting technical indicators 
import csv
import MySQLdb

mydb = MySQLdb.connect(host=h,
                         database=db,
                         user=u,
                         password=pswrd, autocommit=True)
cursor = mySQLconnection.cursor()
csv_data = csv.reader(open("technical_indicators.csv"))
headers = next(csv_data) 
headers
for row in csv_data:
    cursor.execute('INSERT IGNORE INTO technical_analysis(ta_id,symbol,date,sma10,sma30,sma50,ema10,ema30,ema50, \
    macd,macd_signal) VALUES(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)', row)
#close the connection to the database.
mydb.commit()
cursor.close()
print("done")

done
